<a href="https://colab.research.google.com/github/srilamaiti/srilamaiti.github.io/blob/main/agent/multi_agent_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [22]:
!pip install --quiet -U langchain langchain_openai langchainhub
!!pip install --upgrade langgraph
!pip install google
!pip install wikipedia
!pip install duckduckgo-search
!pip install arxiv
!pip install python-dotenv
!pip install faiss-cpu
!pip install -U langchain-openai
!pip install langchain-experimental
!pip install langgraph-supervisor
!pip install langchain-community
!pip install tiktoken
!pip install openai
!pip install langchain_core
!pip install langchain_experimental
!pip install langchain_community
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


# Import Libraries

In [23]:
import gradio as gr
from typing import Annotated
from langchain.llms import OpenAI
import random
import operator
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
from langgraph.graph.message import add_messages
from langchain.agents import AgentType
from langchain.agents import initialize_agent, Tool
from langchain.agents import tool
from typing_extensions import TypedDict
import getpass
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display
from dotenv import load_dotenv
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.environ['GDRIVE_CONFIG_DIR'] = "/content/drive/MyDrive"


Mounted at /content/drive


# Setup

In [3]:
!rm -rf .env
!touch .env
!cat /content/drive/MyDrive/openai_api_key.txt >> .env
!cat /content/drive/MyDrive/langchain_api_key.txt >> .env
!cat /content/drive/MyDrive/tavily_api_key.txt >> .env

In [4]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [5]:
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_PROJECT"] = "LangGraph_01"
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["TAVILY_API_KEY"] = tavily_api_key

#Conversation

In [6]:
llm = OpenAI(temperature=0)
#llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

<ipython-input-6-dfe552c22ea5>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)


In [7]:
@tool
def product_type_search_tool(product_name: str) -> str:
    """
    Use DuckDuckGo to search for the product and summarize its type in a few words.
    """
    try:
        # Initialize DuckDuckGo search tool
        search = DuckDuckGoSearchRun()

        # Perform search
        search_results = search.invoke(f"{product_name} product name")

        # Initialize LLM for summarization
        llm = ChatOpenAI(temperature=0, model="gpt-4")

        # Summarize the search results
        summary_prompt = [
            SystemMessage(content="You are an expert summarizer. Your task is to extract and summarize product types."),
            HumanMessage(content=f"Summarize in 2-3 words the type of product for '{product_name}' based on this information:\n\n{search_results}")
        ]

        # Get summary from LLM
        product_type = llm.invoke(summary_prompt)

        return product_type.content

    except Exception as e:
        return "Unknown"


In [18]:
@tool
def critical_component_search_tool(product_type: str) -> str:
    """Use this tool to perform web searches to find out critical components of a type of product using DuckDuckGo."""
    try:

        # Initialize DuckDuckGo search tool
        search = DuckDuckGoSearchRun()

        # Perform search
        search_results = search.invoke(f"{product_type} product type")

        # Initialize LLM for summarization
        llm = ChatOpenAI(temperature=0, model="gpt-4")

        # Summarize the search results
        summary_prompt = [
            SystemMessage(content="""You are a helful assistant and an expert summarizer. Using the result from first step, please list only the unique most critical functional component names of this type of products Please do not include any specifications or compatability or preferences or impacts?"""),
            HumanMessage(content=f"What are the critical components for '{product_type}' based on this information:\n\n{search_results}")
        ]

        # Get summary from LLM
        product_type = llm.invoke(summary_prompt)

        return product_type.content

    except Exception as e:
        return "Error"

In [10]:
product_type_search_tool.invoke("CMCBL730P1")

'Cordless Brushless Blower'

In [19]:
critical_component_search_tool(product_type_search_tool.invoke("CMCBL730P1"))

'1. Lithium-ion Battery\n2. Motor\n3. Airflow System\n4. Noise Reduction System'

In [29]:
# Agentic Function: Search Product Type and Critical Components
@tool("Search Product Type and Critical Components")
def search_product_type_and_critical_components(product_name: str):
    """Calls both tools and returns the product type and its critical components."""
    product_type = product_type_search_tool(product_name)

    if not product_type or product_type.lower() in ["unknown", "none"]:
        return f"Could not determine the product type for '{product_name}'."

    critical_components = critical_component_search_tool(product_type)

    return f"Product Type: {product_type}\nCritical Components: {critical_components}"

# Chatbot Loop
def chatbot():
    print("🔹 Welcome to the Product Search Chatbot! Type 'quit' to exit.")
    while True:
        user_input = input("\nEnter a product name: ").strip()

        if user_input.lower() in ["quit", "exit"]:
            print("🔹 Goodbye! Chatbot session ended.")
            break

        response = search_product_type_and_critical_components(user_input)
        print("\n" + response)

chatbot()

🔹 Welcome to the Product Search Chatbot! Type 'quit' to exit.

Enter a product name: CMCK497P1

Product Type: Battery String Trimmer Kit
Critical Components: 1. Battery
2. Brushless String Trimmer
3. Cutting Swath
4. Line Diameter
5. Cooling Technology
6. Combi Switch
7. Telescopic Shaft
8. Weed Eaters
9. Weed Wackers
10. Weed Trimmers
11. Grass Trimmers

Enter a product name: CMCN618B

Product Type: Cordless Brad Nailer
Critical Components: 1. Brushless Motor
2. Magazine Capacity
3. Power Source (Battery)
4. LED Lights

Enter a product name: quit
🔹 Goodbye! Chatbot session ended.


In [28]:
# Create a Gradio interface with updated syntax
interface = gr.Interface(
    fn=search_product_type_and_critical_components,
    inputs=gr.Textbox(lines=1, label="product_name"),
    outputs=gr.Textbox(),
    title="Product type and critical component search",
    description="Enter a product number"
)

# Launch the interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fd1167a0a1b718d9fb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
